## Custom Training Loops in Keras

In [1]:
import os
import warnings
import tensorflow as tf 
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.callbacks import Callback
import numpy as np

# Suppress all Python warnings
warnings.filterwarnings('ignore')

# Set TensorFlow log level to suppress warnings and info messages
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# Step 1: Set Up the Environment
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data() 
x_train, x_test = x_train / 255.0, x_test / 255.0 
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(32)


In [2]:
# Step 2: Define the Model

model = Sequential([
    Flatten(input_shape=(28, 28)),
    Dense(128, activation='relu'),
    Dense(10)
])

In [3]:
# Step 3: Define Loss Function and Optimizer

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True) 
optimizer = tf.keras.optimizers.Adam()

In [4]:
# Step 4: Implement the Custom Training Loop

epochs = 2
# train_dataset = train_dataset.repeat(epochs)
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(32)
for epoch in range(epochs):
    print(f'Start of epoch {epoch + 1}')

    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
        with tf.GradientTape() as tape:
            logits = model(x_batch_train, training=True)  # Forward pass
            loss_value = loss_fn(y_batch_train, logits)  # Compute loss

        # Compute gradients and update weights
        grads = tape.gradient(loss_value, model.trainable_weights)
        optimizer.apply_gradients(zip(grads, model.trainable_weights))

        # Logging the loss every 200 steps
        if step % 200 == 0:
            print(f'Epoch {epoch + 1} Step {step}: Loss = {loss_value.numpy()}')

Start of epoch 1
Epoch 1 Step 0: Loss = 2.3743958473205566
Epoch 1 Step 200: Loss = 0.39980998635292053
Epoch 1 Step 400: Loss = 0.1824227124452591
Epoch 1 Step 600: Loss = 0.190804585814476
Epoch 1 Step 800: Loss = 0.21089746057987213
Epoch 1 Step 1000: Loss = 0.48431193828582764
Epoch 1 Step 1200: Loss = 0.1345197856426239
Epoch 1 Step 1400: Loss = 0.26571205258369446
Epoch 1 Step 1600: Loss = 0.23788246512413025
Epoch 1 Step 1800: Loss = 0.15216906368732452
Start of epoch 2
Epoch 2 Step 0: Loss = 0.0945412814617157
Epoch 2 Step 200: Loss = 0.1728932112455368
Epoch 2 Step 400: Loss = 0.11890389025211334
Epoch 2 Step 600: Loss = 0.08480465412139893
Epoch 2 Step 800: Loss = 0.10181252658367157
Epoch 2 Step 1000: Loss = 0.28182512521743774
Epoch 2 Step 1200: Loss = 0.05708758533000946
Epoch 2 Step 1400: Loss = 0.16776081919670105
Epoch 2 Step 1600: Loss = 0.1877271980047226
Epoch 2 Step 1800: Loss = 0.08635355532169342


### Adding Accuracy Metric

In [5]:
import tensorflow as tf 
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense, Flatten 

# Step 1: Set Up the Environment
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Normalize the pixel values to be between 0 and 1
x_train, x_test = x_train / 255.0, x_test / 255.0 

# Create a batched dataset for training
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(32)

In [6]:
# Step 2: Define the Model

model = Sequential([ 
    Flatten(input_shape=(28, 28)),  # Flatten the input to a 1D vector
    Dense(128, activation='relu'),  # First hidden layer with 128 neurons and ReLU activation
    Dense(10)  # Output layer with 10 neurons for the 10 classes (digits 0-9)
])

In [7]:
# Step 3: Define Loss Function, Optimizer, and Metric

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)  # Loss function for multi-class classification
optimizer = tf.keras.optimizers.Adam()  # Adam optimizer for efficient training
accuracy_metric = tf.keras.metrics.SparseCategoricalAccuracy()  # Metric to track accuracy during training

In [8]:
# Step 4: Implement the Custom Training Loop with Accuracy

epochs = 5  # Number of epochs for training

for epoch in range(epochs):
    print(f'Start of epoch {epoch + 1}')
    
    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
        with tf.GradientTape() as tape:
            # Forward pass: Compute predictions
            logits = model(x_batch_train, training=True)
            # Compute loss
            loss_value = loss_fn(y_batch_train, logits)
        
        # Compute gradients
        grads = tape.gradient(loss_value, model.trainable_weights)
        # Apply gradients to update model weights
        optimizer.apply_gradients(zip(grads, model.trainable_weights))
        
        # Update the accuracy metric
        accuracy_metric.update_state(y_batch_train, logits)

        # Log the loss and accuracy every 200 steps
        if step % 200 == 0:
            print(f'Epoch {epoch + 1} Step {step}: Loss = {loss_value.numpy()} Accuracy = {accuracy_metric.result().numpy()}')
    
    # Reset the metric at the end of each epoch
    accuracy_metric.reset_state()

Start of epoch 1
Epoch 1 Step 0: Loss = 2.3632686138153076 Accuracy = 0.1875
Epoch 1 Step 200: Loss = 0.4184502363204956 Accuracy = 0.8347325921058655
Epoch 1 Step 400: Loss = 0.1777428388595581 Accuracy = 0.8687655925750732
Epoch 1 Step 600: Loss = 0.1555330902338028 Accuracy = 0.8844113945960999
Epoch 1 Step 800: Loss = 0.20300383865833282 Accuracy = 0.8966526389122009
Epoch 1 Step 1000: Loss = 0.3931822180747986 Accuracy = 0.9040022492408752
Epoch 1 Step 1200: Loss = 0.15781570971012115 Accuracy = 0.9107774496078491
Epoch 1 Step 1400: Loss = 0.1900905966758728 Accuracy = 0.915662944316864
Epoch 1 Step 1600: Loss = 0.21526694297790527 Accuracy = 0.9188007712364197
Epoch 1 Step 1800: Loss = 0.14350934326648712 Accuracy = 0.922751247882843
Start of epoch 2
Epoch 2 Step 0: Loss = 0.08563232421875 Accuracy = 1.0
Epoch 2 Step 200: Loss = 0.17792275547981262 Accuracy = 0.961442768573761
Epoch 2 Step 400: Loss = 0.10861668735742569 Accuracy = 0.9585411548614502
Epoch 2 Step 600: Loss = 0.04

### Custom Callback for Advanced Logging

In [9]:
import tensorflow as tf 
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense, Flatten 

# Step 1: Set Up the Environment
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Normalize the pixel values to be between 0 and 1
x_train, x_test = x_train / 255.0, x_test / 255.0 

# Create a batched dataset for training
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(32)